## Azure AI Foundry: Agents Service - Deep Research (Preview)

### Environment Setup

In [1]:
# Import required packages
import os
import time
import json
from typing import Optional
from azure.identity import DefaultAzureCredential
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import (
    DeepResearchTool,
    MessageRole,
    ThreadMessage,
    SubmitToolApprovalAction,
    ToolApproval
)

In [2]:
# Set environment variables
PROJECT_ENDPOINT = os.getenv("AZURE_FOUNDRY_PROJECT_ENDPOINT")
BING_SEARCH_CONN_ID = os.getenv("AZURE_FOUNDRY_BINGSEARCH_CONNECTION")
O3DR_DEPLOYMENT = os.getenv("AZURE_FOUNDRY_O3DR_DEPLOYMENT", "o3-deep-research")
GPT4O_DEPLOYMENT = os.getenv("AZURE_FOUNDRY_GPT40_DEPLOYMENT", "gpt-4o")

In [ ]:
# User prompt for agentic task
USER_PROMPT = "Give me the latest research into quantum computing over the last year."

### Helper Functions

In [4]:
# Function to fetch and print new agent responses
def fetch_and_print_new_agent_response(thread_id: str, agents_client: AgentsClient, last_message_id: Optional[str] = None) -> Optional[str]:
    response = agents_client.messages.get_last_message_by_role(
        thread_id = thread_id,
        role = MessageRole.AGENT,
    )
    if not response or response.id == last_message_id:
        return last_message_id  # No new content

    print("\nAgent response:")
    print("\n".join(t.text.value for t in response.text_messages))

    for ann in response.url_citation_annotations:
        print(f"URL Citation: [{ann.url_citation.title}]({ann.url_citation.url})")

    return response.id

In [5]:
# Function to create a research summary markdown file
def create_research_summary(message: ThreadMessage, filepath: str = "research_summary.md") -> None:
    if not message:
        print("No message content provided, cannot create research summary.")
        return

    with open(filepath, "w", encoding="utf-8") as fp:
        # Write text summary
        text_summary = "\n\n".join([t.text.value.strip() for t in message.text_messages])
        fp.write(text_summary)

        # Write unique URL citations, if present
        if message.url_citation_annotations:
            fp.write("\n\n## References\n")
            seen_urls = set()
            for ann in message.url_citation_annotations:
                url = ann.url_citation.url
                title = ann.url_citation.title or url
                if url not in seen_urls:
                    fp.write(f"- [{title}]({url})\n")
                    seen_urls.add(url)

    print(f"Research summary written to '{filepath}'.")

### Agent Setup

In [6]:
# Setup AI Foundry client
agents_client = AgentsClient(
    endpoint = PROJECT_ENDPOINT,
    credential = DefaultAzureCredential()
)

In [7]:
# Initialise a Deep Research tool with Bing Connection ID and o3-deep-research model
deep_research_tool = DeepResearchTool(
    bing_grounding_connection_id = BING_SEARCH_CONN_ID,
    deep_research_model = O3DR_DEPLOYMENT,
)

In [8]:
# Create AI agent with a Deep Research tool
agent = agents_client.create_agent(
    model = GPT4O_DEPLOYMENT,
    name = "deep-research-agent",
    instructions = "You are a helpful Agent that assists in researching scientific topics.",
    tools = deep_research_tool.definitions
)

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_nxOh68SD1wgwtJQqc4p8Xc5w


### Agent Run

In [9]:
# Create thread and message
thread = agents_client.threads.create()
print(f"Created thread, ID: {thread.id}")

message = agents_client.messages.create(
    thread_id = thread.id,
    role = "user",
    content = USER_PROMPT
)
print(f"Created message, ID: {message.id}")

Created thread, ID: thread_4LDD4NSy2SpQsjnm146qnxP9
Created message, ID: msg_77wG9uzFOhKyvg1q8QY2Q1Pm


In [10]:
# Poll the run as long as run status is queued or in progress
print(f"Start processing the message... this may take a few minutes to finish. Be patient!")        

run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id)
last_message_id = None

while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action" and isinstance(
        run.required_action, SubmitToolApprovalAction
    ):
        tool_calls = run.required_action.submit_tool_approval.tool_calls
        if not tool_calls:
            print("Run requested tool approval but no tool calls were provided; cancelling run.")
            agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
            break

        approvals = [ToolApproval(tool_call_id=tool_call.id, approve=True) for tool_call in tool_calls]
        print(f"Submitting tool approvals for {len(approvals)} tool calls.")
        run = agents_client.runs.submit_tool_outputs(
            thread_id = thread.id,
            run_id = run.id,
            tool_approvals = approvals,
        )
        continue

    last_message_id = fetch_and_print_new_agent_response(
        thread_id = thread.id,
        agents_client = agents_client,
        last_message_id = last_message_id,
    )
    print(f"Run status: {run.status}")

print(f"Run finished with status: {run.status}, ID: {run.id}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Start processing the message... this may take a few minutes to finish. Be patient!
Run status: RunStatus.IN_PROGRESS

Agent response:
I'll investigate the latest research developments in quantum computing over the last year and provide a summary of significant advancements, breakthroughs, and milestones. I will also include references to the research sources.

Title: Latest Research in Quantum Computing (2024-2025)

 Starting deep research... 

Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS

Agent response:
cot_summary: **Engaging with quantum advancements**

I’m gathering insights on significant research over the past year, covering quantum algorithms, hardware, e

In [11]:
# Fetch the final message from the agent in the thread and create a research summary
final_message = agents_client.messages.get_last_message_by_role(
    thread_id = thread.id,
    role = MessageRole.AGENT
)

if final_message:
    create_research_summary(final_message)

Research summary written to 'research_summary.md'.


### Housekeeping

In [12]:
# Clean up the created resources
print("--- Cleaning up resources ---")

agents_client.threads.delete(thread.id)
print(f"Deleted thread: {thread.id}")

agents_client.delete_agent(agent.id)
print(f"Deleted agent: {agent.id}")

--- Cleaning up resources ---
Deleted thread: thread_4LDD4NSy2SpQsjnm146qnxP9
Deleted agent: asst_nxOh68SD1wgwtJQqc4p8Xc5w
